<div style="color:#485053;
           display:fill;
           border-radius:0px;
           background-color:#86C2DE;
           font-size:200%;
           padding-left:400px; 
           font-family:Verdana;
           font-weight:600; 
           letter-spacing:0.5px;
           ">

<p style="padding: 15px;
          color:white;
          text-align: center;">

 HR ANALYTICS
</p>
</div>

![](https://www.thehrdigest.com/wp-content/uploads/2020/01/hr-analytics.jpg)

<div style="color:#262A49;
           display:fill;
           border-radius:0px;
           background-color:#C2C4E2;
           font-size:177%;
           font-family:Verdana;
           font-weight:200; 
           padding-left: 450px; 
           letter-spacing:0.5px;
           ">

<p style="padding: 15px;
          color:white;
          text-align: center;">

 LIBRARIES
</p>
</div>

# Import Libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
from keras.layers import Dense, BatchNormalization, Dropout, LSTM
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from keras import callbacks

<div style="color:#262A49;
           display:fill;
           border-radius:0px;
           background-color:#C2C4E2;
           font-size:177%;
           font-family:Verdana;
           font-weight:200; 
           padding-left: 430px; 
           letter-spacing:0.5px;
           ">

<p style="padding: 15px;
          color:white;
          text-align: center;">

 LOADING DATA
</p>
</div>



In [ ]:
import pandas as pd
data=pd.read_csv("../input/hr-analytics-job-change-of-data-scientists/aug_train.csv")
data.head()

**About the data :**

This dataset designed to understand the factors that lead a person to leave current job for HR researches too. By model(s) that uses the current credentials,demographics,experience data you will predict the probability of a candidate to look for a new job or will work for the company, as well as interpreting affected factors on employee decision.

The whole data divided to train and test . Target isn't included in test but the test target values data file is in hands for related tasks. A sample submission correspond to enrollee_id of test set provided too with columns : enrollee _id , target

In [ ]:
data.info()

**Point to note:**

* There are missing values in the dataset
* Dataset includes categorical and numerical values

<div style="color:#485053;
           display:fill;
           border-radius:0px;
           background-color:#86C2DE;
           font-size:177%;
           padding-left:300px; 
           font-family:Verdana;
           font-weight:200; 
           letter-spacing:0.5px;
           ">

<p style="padding: 15px;
          color:white;
          text-align: center;">

 DATA VISUALIZATION AND CLEANING
</p>
</div>

# **Count plot of target column**

In [ ]:
#first of all let us evaluate the target and find out if our data is imbalanced or not
cols= ["#86C2DE","#FDB355"]
sns.countplot(x= data["target"], palette= cols)

In [ ]:
# Correlation amongst numeric attributes
corrmat = data.corr()
cmap = sns.diverging_palette(200,-10,s=50, l=75, n=6, as_cmap=True)
plt.subplots(figsize=(7,7))
sns.heatmap(corrmat,cmap= cmap,annot=True, square=True)

In [ ]:
# plot male and female employees in the company

cols= ["#86C2DE","#FDB355"]
sns.countplot(x= data["gender"], palette= cols)

**Categorical variables**

In [ ]:
# Get list of categorical variables
s = (data.dtypes == "object")
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

In [ ]:
# Missing values in categorical variables

for i in object_cols:
    print(i, data[i].isnull().sum())

In [ ]:
# Filling missing values with mode of the column in value

for i in object_cols:
    data[i].fillna(data[i].mode()[0], inplace=True)

**Numerical variables**
* Filling numerical variables with median of the column value

In [ ]:
# Get list of numeric variables
t = (data.dtypes == "float64")
num_cols = list(t[t].index)

print("Numeric variables:")
print(num_cols)

In [ ]:
# Missing values in numeric variables

for i in num_cols:
    print(i, data[i].isnull().sum())

There are no missing values in numeric data. Hence no need to fill.

In [ ]:
#Evauating education_level over training hours
colours = ["#D0DBEE", "#C2C4E2", "#EED4E5", "#D1E6DC", "#BDE2E2"]
plt.figure(figsize=(12,8))
Days_of_week=sns.barplot(x=data['education_level'],y="training_hours",data=data, ci =None,palette = colours)
Days_of_week.set_title("Education Level over training hours")
Days_of_week.set_ylabel("Training Hours")
Days_of_week.set_xlabel("Education Level")

<div style="color:#262A49;
           display:fill;
           border-radius:0px;
           background-color:#86C2DE;
           font-size:177%;
           font-family:Verdana;
           font-weight:200; 
           padding-left: 400px; 
           letter-spacing:0.5px;
           ">

<p style="padding: 15px;
          color:white;
          text-align: center;">

DATA PREPROCESSING
</p>
</div>

**Label encoder the categorical value**

In [ ]:
# Apply label encoder to each column with categorical data
label_encoder = LabelEncoder()
for i in object_cols:
    data[i] = label_encoder.fit_transform(data[i])
    
data.info()

In [ ]:
# Prepairing attributes of scale data

features = data.drop(['target'], axis=1) # dropping target and extra columns

target = data['target']

#Set up a standard scaler for the features
col_names = list(features.columns)
s_scaler = preprocessing.StandardScaler()
features = s_scaler.fit_transform(features)
features = pd.DataFrame(features, columns=col_names) 

features.describe().T

In [ ]:
#Detecting outliers
#looking at the scaled features
colours = ["#D0DBEE", "#C2C4E2", "#EED4E5", "#D1E6DC", "#BDE2E2"]
plt.figure(figsize=(20,10))
sns.boxenplot(data = features,palette = colours)
plt.xticks(rotation=90)
plt.show()

No outliers to remove!

<div style="color:#262A49;
           display:fill;
           border-radius:0px;
           background-color:#86C2DE;
           font-size:177%;
           font-family:Verdana;
           font-weight:200; 
           padding-left: 400px; 
           letter-spacing:0.5px;
           ">

<p style="padding: 15px;
          color:white;
          text-align: center;">

MODEL BUILDING
</p>
</div>

In [ ]:
X = features
y = target

# Splitting test and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

X.shape

In [ ]:
#Early stopping
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# Initialising the NN
model = Sequential()

# layers
model.add(Dense(units = 25, kernel_initializer = 'uniform', activation = 'relu', input_dim = 13))
model.add(Dense(units = 25, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 5, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Train the ANN
history = model.fit(X_train, y_train, batch_size = 32, epochs = 150,callbacks=[early_stopping], validation_split=0.2)

In [ ]:
history_df = pd.DataFrame(history.history)

plt.plot(history_df.loc[:, ['loss']], "#BDE2E2", label='Training loss')
plt.plot(history_df.loc[:, ['val_loss']],"#C2C4E2", label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")

plt.show()

In [ ]:
history_df = pd.DataFrame(history.history)

plt.plot(history_df.loc[:, ['accuracy']], "#BDE2E2", label='Training accuracy')
plt.plot(history_df.loc[:, ['val_accuracy']], "#C2C4E2", label='Validation accuracy')

plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

<div style="color:#262A49;
           display:fill;
           border-radius:0px;
           background-color:#86C2DE;
           font-size:177%;
           font-family:Verdana;
           font-weight:200; 
           padding-left: 400px; 
           letter-spacing:0.5px;
           ">

<p style="padding: 15px;
          color:white;
          text-align: center;">

CONCLUSION
</p>
</div>

In [ ]:
# Predicting the test set results
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
# confusion matrix
import numpy as np
cmap1 = sns.diverging_palette(260,-10,s=50, l=75, n=5, as_cmap=True)
plt.subplots(figsize=(12,8))
cf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(cf_matrix/np.sum(cf_matrix), cmap = cmap1, annot = True, annot_kws = {'size':15})

In [ ]:
print(classification_report(y_test, y_pred))

[Lets Connect on LinkedIn!](https://www.linkedin.com/in/krupa-gajjar-02011999/)